# 1. Introduction 

This notebook will walk you through some steps to find a suitable location to open a restaurant in Toronto, Canada based on Foursquare venues database. 


## Audience

People who are interested in opening a restaurant in Toronto

## Problem to solve: how to find a suitable place to open my  restaurant ? 

### Things to consider 

Besides menu, food quality, special recipe, space size or decoration concept, location plays a very important part to your success as a restaurant owner. There's no silver bullet to solve the "where 's the best place for my restaurent?", of course, but we can give you some insights that you then choose it yourself. Because at the end of the day, it all depends on your restaurant concept and your targeted customer. 

Basically, our location research should cover following things:

#### a.  Visibility 

Easy to notice, easy to find ( both on real life and location mobile applitions ). In your early days, a nearby popular venue should be very helpful.  

#### b.  Accessibility

It should be easy for people to get there by different kinds of transportation methods. Your customer base will be bigger if you have this advantage. Even when you own a car, you still want to use public transportation service sometimes to avoid the parking issue ( or you're just too lazy ). 

On the other side, talking about accessibility, a restaurant should be in a convenient place for their supplier ( Eg: raw material ) as well. You don't want to cross half way around the city to buy some basic ingredients in urgent case. 

#### c. Parking lot issue

Very important. No matter how good your the food is, If I have to spend more than 15 mins to find the parking lot or walk for more 500m before entering your restaurant, I'll be very annoying after all. 
A place near by a parking lot is perfect but a neighbor hotel should be ok to make a deal with you to share their parking service. 

#### d. Potential customer base nearby

A restaurant which server dinner ( casual or fine ) should be close to residence area. Bistro should be close to office. Crowded places like metro, train station, bus stop..are good places for fast food. 
Or after all, in general, more people, more traffic, more chances that someone will visit your restaurent. You definitely don't want to open your restaurant in a cave. 

#### e. The competitors nearby 

If there are 20 restaurants with the same concept as yours within a 1 km radius already, you must be very ambitious and invest much more money/effort to get your bite. On the other hand, if there's no such restaurant of the same type yet, maybe peole in this area don't really like such kind of food ? 

Knowing the competitors nearby can be very helpful for your business strategy.



# 2. Data
### Propose solution by using foursquare location database

Follow our previous research about [Toronto neighborhood](https://github.com/kembox/Coursera_Capstone/blob/master/toronto_neighborhoods.ipynb) , we'll reuse location data for Toronto borough/neighborhood to feed to foursquare [search API](https://developer.foursquare.com/docs/api/venues/search) with specific [categoryId](https://developer.foursquare.com/docs/resources/categories) to get following info for each area ( borough that has its own postal code ): 

- Visible venue: number of intersection within 500m radius within each area. The bigger the better. ( More places with foot traffic, more options for you to choose a place that can attract your customer well ) 
- Highly accessible venue: number of public transportation within 500m radius. ( Same reason as above: more traffic, more customer, most likely ). The good place should also have a market / Supermarket / grocery store / gourmet ...  in <= 3km nearby. The more of those places exist within the area we're examining, the more likely we'll choose that location. 
- Parking lot venue: number of parking lot and Hotel/Office category nearby ( As we can deal with them to share their parking service ) , withint 500m radius. Again, bigger better. 
- Your customer base info in this area: info about places nearby that you can expect customer from ( Eg: Residence, Hotel, Office, MovieTheater, Supermarket, Shopping Mall, College & University...  ) 
- Competitors nearby: grab info about restaurent of the same type as yours ( for example, Asian food ) in each area. Based on the number / distribution of our competitor, we can decide about strategic steps then. 
 

Base on the data mentioned above, we can run a classification model to divide Toronto borough into different types based on their "suitable level" of being our restaurant location. 
Most likely that the location that can satisfy all our factors will have high cost or even ran out of available place for rent. Classify them well will give us more options to choose to fit our initial budget.

# 3. Methodology
Main component of the report where we do the job for collecting and processing data explained above, fit them to a classification model and visualize the results so we can use it to get answer of our business problem

## 3.1 Collect and process data

#### Install and  Import needed lib 

In [8]:
#Install and import BeautifulSoup library to parse XML data above
!pip3 --quiet install bs4

#import needed lib
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import requests
import numpy as np
from sklearn.cluster import KMeans 

import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim

!pip3 --quiet install folium
import folium

import foursquare

#### Download Toronto location data 
The data which was collected from wikipedia and processed by method explained [here](https://github.com/kembox/Coursera_Capstone/blob/master/toronto_neighborhoods.ipynb)

In [6]:
!wget https://raw.githubusercontent.com/kembox/Coursera_Capstone/master/toronto_data.csv -O toronto_data.csv --quiet

In [7]:
#Load data to a dataframe
toronto_data=pd.read_csv('toronto_data.csv',index_col=None)
toronto_data.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
